Обробка зображення на навченій моделі

In [ ]:
from ultralytics import YOLO
import cv2

image = r"C:\Users\dimag\Desktop\6268353d59de8f00070b329c.tif"

img = cv2.imread(image)
height, width = img.shape[:2]
max_side = max(width, height)
if max_side > 4096:
    max_side = 1024

model = YOLO(r"\\RANOBEWORLD\RanobeWorld-Disk\Files\Datasets\xview_datasets\yolo_xview_trained_model_newDataset_50ep.pt")
results = model.predict(source=image, imgsz=max_side, conf=0.2)

for box in results[0].boxes:
    x1, y1, x2, y2 = map(int, box.xyxy[0]) 
    class_name = model.names[int(box.cls[0])]
    confidence = box.conf[0]  

    label = f"{class_name} {confidence:.2f}"

    cv2.rectangle(img, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=1)  # Тонка рамка
    cv2.putText(img, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)  # Невеликий шрифт


cv2.imwrite(r"C:\Users\dimag\Desktop\4789789.jpg", img)


image 1/1 C:\Users\dimag\Desktop\6268353d59de8f00070b329c.tif: 1024x1024 5 Barges, 8 Ferrys, 2 Container Ships, 3 Shipping container lots, 3577.7ms
Speed: 14.9ms preprocess, 3577.7ms inference, 11.0ms postprocess per image at shape (1, 3, 1024, 1024)


True

Тайлова обробка зображення

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict

model = YOLO(r"C:\Users\dimag\Desktop\yolo_xview_trained_model_newDataset_50ep.pt")

tile_size = 2048
overlap = 0.1
confidence_threshold = 0.1

def get_tiles(image, tile_size, overlap):
    height, width = image.shape[:2]
    step = int(tile_size * (1 - overlap))
    tiles = []
    positions = []
    
    for y in range(0, height, step):
        for x in range(0, width, step):
            x_end = min(x + tile_size, width)
            y_end = min(y + tile_size, height)
            tile = image[y:y_end, x:x_end]
            tiles.append(tile)
            positions.append((x, y, x_end, y_end))
    
    return tiles, positions


def process_tile(tile, position, model):
    x_offset, y_offset = position[:2]
    results = model.predict(source=tile, imgsz=tile.shape[1], conf=confidence_threshold)
    detections = []
    
    for box in results[0].boxes:
        confidence = box.conf[0]
        if confidence >= confidence_threshold:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            class_id = int(box.cls[0])
            detections.append({
                "coords": (x1 + x_offset, y1 + y_offset, x2 + x_offset, y2 + y_offset),
                "class_id": class_id,
                "confidence": confidence
            })
    
    return detections


def process_large_image(image_path):

    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    tiles, positions = get_tiles(image, tile_size, overlap)

    all_detections = []

    for tile, position in zip(tiles, positions):
        detections = process_tile(tile, position, model)
        all_detections.extend(detections)

    unique_detections = []
    counted_coords = set()
    for detection in all_detections:
        x1, y1, x2, y2 = detection["coords"]
        if (x1, y1, x2, y2) not in counted_coords:
            unique_detections.append(detection)
            counted_coords.add((x1, y1, x2, y2))

    class_counts = defaultdict(int)
    for detection in unique_detections:
        x1, y1, x2, y2 = detection["coords"]
        class_id = detection["class_id"]
        class_name = model.names[class_id]
        confidence = detection["confidence"]

        cv2.rectangle(image, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=1)
        label = f"{class_name} {confidence:.2f}"
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

        class_counts[class_name] += 1


    output_path = r"C:\Users\dimag\Desktop\output_with_detections.jpg"
    cv2.imwrite(output_path, image)

    total_count = len(unique_detections)
    print(f"Общее количество объектов: {total_count}")
    for class_name, count in class_counts.items():
        print(f"{class_name}: {count}")

    return output_path

image_path = r"C:\Users\dimag\Desktop\олд\24bfc3f8-1ff1-47dd-b06a-7e8f95a57c5f.tif"
output_image_path = process_large_image(image_path)
print(f"Результат збережено в {output_image_path}")


In [ ]:
import os
import cv2
import numpy as np
from ultralytics import YOLO
from collections import defaultdict


model = YOLO(r"C:\Users\dimag\Desktop\detect_results_dota_big_3ep\obb\train6\weights\best.pt")

tile_size = 2048
overlap = 0.2  
confidence_threshold = 0.1
output_tile_dir = os.path.join(os.getcwd(), "test_tiles")
os.makedirs(output_tile_dir, exist_ok=True)


def get_tiles_with_overlap(image, tile_size, overlap):
    height, width = image.shape[:2]
    step = int(tile_size * (1 - overlap))
    tiles = []
    positions = []
    
    for y in range(0, height, step):
        for x in range(0, width, step):
            x_end = min(x + tile_size, width)
            y_end = min(y + tile_size, height)
            tile = image[y:y_end, x:x_end]
            tiles.append(tile)
            positions.append((x, y, x_end, y_end))
    
    return tiles, positions


def process_tile(tile, position, model):
    x_offset, y_offset = position[:2]
    results = model.predict(source=tile, imgsz=tile_size, conf=confidence_threshold)
    detections = []
    
    for box in results[0].boxes:
        confidence = box.conf[0]
        if confidence >= confidence_threshold:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            class_id = int(box.cls[0])

            detections.append({
                "coords": (x1 + x_offset, y1 + y_offset, x2 + x_offset, y2 + y_offset),
                "class_id": class_id,
                "confidence": confidence
            })
    
    return detections


def process_large_image(image_path):
    image = cv2.imread(image_path)
    height, width = image.shape[:2]

    tiles, positions = get_tiles_with_overlap(image, tile_size, overlap)

    all_detections = []

    for i, (tile, position) in enumerate(zip(tiles, positions)):
        detections = process_tile(tile, position, model)
        all_detections.extend(detections)

        tile_path = os.path.join(output_tile_dir, f"tile_{i}.jpg")
        for detection in detections:
            x1, y1, x2, y2 = detection["coords"]
            cv2.rectangle(tile, (x1 - position[0], y1 - position[1]), 
                          (x2 - position[0], y2 - position[1]), 
                          color=(0, 255, 0), thickness=1)
            label = f"{model.names[detection['class_id']]} {detection['confidence']:.2f}"
            cv2.putText(tile, label, (x1 - position[0], y1 - position[1] - 10), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)
        cv2.imwrite(tile_path, tile)

    unique_detections = []
    counted_coords = set()
    for detection in all_detections:
        x1, y1, x2, y2 = detection["coords"]
        bbox_key = (round(x1 / tile_size), round(y1 / tile_size), 
                    round(x2 / tile_size), round(y2 / tile_size))
        if bbox_key not in counted_coords:
            unique_detections.append(detection)
            counted_coords.add(bbox_key)

    class_counts = defaultdict(int)
    for detection in unique_detections:
        x1, y1, x2, y2 = detection["coords"]
        class_id = detection["class_id"]
        class_name = model.names[class_id]
        confidence = detection["confidence"]

        cv2.rectangle(image, (x1, y1), (x2, y2), color=(0, 255, 0), thickness=1)
        label = f"{class_name} {confidence:.2f}"
        cv2.putText(image, label, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (255, 255, 255), 1)

        class_counts[class_name] += 1

    output_image_path = r"C:\Users\dimag\Desktop\output_with_detections.jpg"
    cv2.imwrite(output_image_path, image)

    total_count = len(unique_detections)
    print(f"Общее количество объектов: {total_count}")
    for class_name, count in class_counts.items():
        print(f"{class_name}: {count}")

    return output_image_path

image_path = r"C:\Users\dimag\Desktop\24bfc3f8-1ff1-47dd-b06a-7e8f95a57c5f.tif"
output_image_path = process_large_image(image_path)
print(f"Результат сохранен в {output_image_path}")



0: 2048x2048 (no detections), 17258.0ms
Speed: 34.4ms preprocess, 17258.0ms inference, 22.0ms postprocess per image at shape (1, 3, 2048, 2048)


TypeError: 'NoneType' object is not iterable